<a href="https://colab.research.google.com/github/Andyfer004/Lab-8IA/blob/main/LAB8IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LAB 8 - Inteligencia Artificial  
## Catedrático: Alberto Suriano  
### Estudiantes:  
- Andy Fuentes 22944
- Davis Roldán  22672
- Diederich Solís 22952



## Tasks 1 - Teoría

### 1. Investigar el algoritmo AC-3 y su relación con el algoritmo de backtracking search

AC-3 (Arc Consistency Algorithm 3) es un algoritmo utilizado en problemas de satisfacción de restricciones (CSP) para reducir los dominios de las variables antes o durante la búsqueda. Lo que hace es asegurarse de que todas las variables estén *arc-consistent*, es decir, que para cada valor de una variable exista al menos un valor compatible en las variables relacionadas.

Su relación con el algoritmo de backtracking search es que AC-3 puede utilizarse como una forma de **pre-procesamiento** o como **heurística** durante la búsqueda. Esto permite reducir significativamente el espacio de búsqueda, ya que elimina valores inconsistentes antes de intentar asignaciones, disminuyendo así el número de retrocesos (backtracks) necesarios.



### 2. Defina en sus propias palabras el término “Arc Consistency”

Una variable es *arc-consistent* con respecto a otra si, para cada valor posible en su dominio, hay al menos un valor en el dominio de la otra variable que satisface la restricción entre ambas. Es decir, no hay valores sin apoyo en las variables relacionadas. El objetivo es que no existan valores que, al asignarse, hagan inevitable una falla más adelante.

## Task 2 - CSP con Backtracking, Beam y Local Search

En este ejercicio, resolveremos el problema de calendarización de 7 exámenes para 4 estudiantes (con restricciones) utilizando tres algoritmos diferentes: Backtracking Search, Beam Search y Local Search.



### 1. Definir las variables

**a. Variables:**  
- Representamos cada examen con una variable: `E1, E2, E3, E4, E5, E6, E7`.

**b. Dominio de cada variable:**  
- Los días posibles son: `Lunes`, `Martes` y `Miércoles`.

#### Código: Definición de Variables

In [1]:
# Definición de variables y dominio

from itertools import combinations

# Variables: Exámenes
exams = ["E1", "E2", "E3", "E4", "E5", "E6", "E7"]

# Dominio: Días posibles
days = ["Lunes", "Martes", "Miércoles"]

# Definición de estudiantes y sus exámenes (incluye cursos compartidos)
students = {
    "Andy":      ["E1", "E2", "E3"],
    "Davis":     ["E3", "E4", "E5"],
    "Diederich": ["E5", "E6", "E7"],
    "Shared":    [("E2", "E4"), ("E1", "E6")]
}

### 2. Definir las restricciones

**a. Restricciones:**  
- **Días distintos para cada examen:** Cada examen se realiza en un día distinto.  
- **Un examen por día por estudiante:** Ningún estudiante debe tener más de un examen en el mismo día.  
- **Cursos compartidos:** Los estudiantes que toman el mismo curso (o exámenes relacionados) no pueden tener exámenes el mismo día.

En el código, la función `is_valid` se encargará de verificar estas restricciones.

In [2]:
# Función para verificar si una asignación es válida según las restricciones
def is_valid(assignment, exam, day):
    temp = assignment.copy()
    temp[exam] = day

    # Verificar para cada grupo de exámenes (estudiantes/cursos) que no haya dos asignados al mismo día
    for group in students.values():
        # Si el grupo es una lista de exámenes
        if isinstance(group, list):
            for a, b in combinations(group, 2):
                if a in temp and b in temp and temp.get(a) == temp.get(b):
                    return False
        # Si el grupo es una lista de tuplas (exámenes compartidos)
        else:
            for a, b in group:
                if a in temp and b in temp and temp.get(a) == temp.get(b):
                    return False
    return True

### 3. Implementar el algoritmo de Backtracking Search

**a. Implementación:**  
Utilizaremos un enfoque recursivo en el que asignamos días a los exámenes uno por uno.  
**b. Retroceso:**  
Si se viola alguna restricción, se deshace la asignación (retroceso) y se prueba otro valor.

#### Código: Backtracking Search

In [3]:
import time

def backtrack(assignment):
    # Si se han asignado todos los exámenes, se retorna la asignación
    if len(assignment) == len(exams):
        return assignment

    # Seleccionamos el siguiente examen sin asignar
    unassigned = [e for e in exams if e not in assignment]
    exam = unassigned[0]

    # Probar asignar cada día posible
    for day in days:
        if is_valid(assignment, exam, day):
            assignment[exam] = day
            result = backtrack(assignment)
            if result:
                return result
            # Retroceso: remover la asignación si no conduce a solución
            del assignment[exam]
    return None

start = time.time()
solution_bt = backtrack({})
end = time.time()

print("Backtracking Solution:", solution_bt)
print("Tiempo:", round(end - start, 4), "s")

Backtracking Solution: {'E1': 'Lunes', 'E2': 'Martes', 'E3': 'Miércoles', 'E4': 'Lunes', 'E5': 'Martes', 'E6': 'Lunes', 'E7': 'Miércoles'}
Tiempo: 0.0003 s


### 4. Implementar el algoritmo de Beam Search

**a. Implementación:**  
Se exploran múltiples estados parciales y se conservan los `k` mejores (con menos conflictos).  
**b. Heurística:**  
La función `count_conflicts` cuenta los conflictos de una asignación parcial, y se utiliza para seleccionar los estados más prometedores.

#### Código: Beam Search (con beam width = 2)

In [4]:
import heapq

# Función para contar los conflictos en una asignación
def count_conflicts(assignment):
    conflicts = 0
    for group in students.values():
        if isinstance(group, list):
            for a, b in combinations(group, 2):
                if a in assignment and b in assignment and assignment.get(a) == assignment.get(b):
                    conflicts += 1
        else:
            for a, b in group:
                if a in assignment and b in assignment and assignment.get(a) == assignment.get(b):
                    conflicts += 1
    return conflicts

def beam_search(k=2):
    beam = [({}, 0)]  # Cada elemento es una tupla: (estado parcial, número de conflictos)

    for exam in exams:
        new_beam = []
        for state, _ in beam:
            for day in days:
                new_state = state.copy()
                new_state[exam] = day
                c = count_conflicts(new_state)
                new_beam.append((new_state, c))
        # Conservamos los k mejores estados
        beam = heapq.nsmallest(k, new_beam, key=lambda x: x[1])

        # Si se encontró una solución completa sin conflictos, salimos
        if any(len(b[0]) == len(exams) and b[1] == 0 for b in beam):
            break

    best = min(beam, key=lambda x: x[1])
    return best

start = time.time()
solution_beam, conflicts_beam = beam_search(k=2)
end = time.time()

print("Beam Search Solution:", solution_beam)
print("Conflictos:", conflicts_beam)
print("Tiempo:", round(end - start, 4), "s")

Beam Search Solution: {'E1': 'Lunes', 'E2': 'Martes', 'E3': 'Miércoles', 'E4': 'Lunes', 'E5': 'Martes', 'E6': 'Lunes', 'E7': 'Miércoles'}
Conflictos: 0
Tiempo: 0.0004 s


### 5. Implementar el algoritmo de Local Search

**a. Implementación:**  
Se parte de una solución aleatoria y se mejora iterativamente buscando soluciones vecinas.  
**b. Heurística:**  
Se evalúan los vecinos mediante el número de conflictos, y se selecciona el vecino con la menor cantidad de conflictos.

#### Código: Local Search (Hill Climbing con reinicios aleatorios)

In [5]:
import random

def generate_random_solution():
    return {e: random.choice(days) for e in exams}

def get_neighbors(solution):
    neighbors = []
    for exam in exams:
        for day in days:
            if solution[exam] != day:
                new_sol = solution.copy()
                new_sol[exam] = day
                neighbors.append(new_sol)
    return neighbors

def local_search(max_restarts=20, max_steps=100):
    for _ in range(max_restarts):
        current = generate_random_solution()
        for _ in range(max_steps):
            neighbors = get_neighbors(current)
            next_solution = min(neighbors, key=count_conflicts)
            # Si se mejora la solución, se actualiza
            if count_conflicts(next_solution) < count_conflicts(current):
                current = next_solution
            else:
                break
        # Si no hay conflictos, se retorna la solución
        if count_conflicts(current) == 0:
            return current
    return None

start = time.time()
solution_local = local_search()
end = time.time()

print("Local Search Solution:", solution_local)
print("Conflictos:", count_conflicts(solution_local) if solution_local else "No encontrada")
print("Tiempo:", round(end - start, 4), "s")

Local Search Solution: {'E1': 'Martes', 'E2': 'Miércoles', 'E3': 'Lunes', 'E4': 'Miércoles', 'E5': 'Martes', 'E6': 'Lunes', 'E7': 'Miércoles'}
Conflictos: 0
Tiempo: 0.0006 s


### Comparación de Resultados y Conclusiones

| Algoritmo         | Tiempo (s) | Conflictos | Solución Válida |
|-------------------|------------|------------|------------------|
| Backtracking      | 0.0003     | 0          | ✅               |
| Beam Search (k=2) | 0.0004     | 0          | ✅               |
| Local Search      | 0.0006     | 0          | ✅               |

**Conclusiones:**

- **Backtracking** encontró una solución válida en el menor tiempo y garantiza exactitud. Es ideal para problemas pequeños o donde se requiere certeza.
- **Beam Search** fue eficiente y encontró una solución válida sin conflictos. Su rendimiento puede variar según el tamaño del beam y la heurística usada.
- **Local Search** también resolvió el problema sin errores, aunque su tiempo fue ligeramente mayor. Es útil en espacios grandes donde una solución rápida aproximada es suficiente.

Los tres algoritmos funcionaron correctamente en este caso. La elección depende del tamaño del problema y la necesidad de precisión o eficiencia.